# Initialization

In [ ]:
#Pip install required dependencies

!pip install ipykernel
!pip install imagej
!pip install pyjnius
!pip install pyimagej
!pip install matplotlib

In [2]:
#Import required packages

import os
import sys
import imagej
import scipy.ndimage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scyjava as sj

from jpype import JInt
from imagej._java import jc

In [3]:
# initialize Fiji endpoint (ver 2.9.0) in interactive mode
import imagej
ij = imagej.init('sc.fiji:fiji:2.9.0')
print(f"ImageJ2/Fiji version: {ij.getVersion()}")

ImageJ2/Fiji version: 2.9.0/1.53t


In [4]:
# load legacy ImageJ resources
DirectoryChooser = sj.jimport('ij.io.DirectoryChooser')
GenericDialog = sj.jimport('ij.gui.GenericDialog')
ResultsTable = sj.jimport('ij.measure.ResultsTable')
TextWindow = sj.jimport('ij.text.TextWindow')

In [5]:
# load ImageJ2/Fiji resources
Model = sj.jimport('fiji.plugin.trackmate.Model')
Settings = sj.jimport('fiji.plugin.trackmate.Settings')
TrackMate = sj.jimport('fiji.plugin.trackmate.TrackMate')
DogDetectorFactory = sj.jimport('fiji.plugin.trackmate.detection.DogDetectorFactory')
SelectionModel = sj.jimport('fiji.plugin.trackmate.SelectionModel')
SimpleSparseLAPTrackerFactory = sj.jimport('fiji.plugin.trackmate.tracking.sparselap.SimpleSparseLAPTrackerFactory')
LAPUtils = sj.jimport('fiji.plugin.trackmate.tracking.LAPUtils')
LabelImgExporter = sj.jimport('fiji.plugin.trackmate.action.LabelImgExporter')
DisplaySettingsIO = sj.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
FeatureFilter = sj.jimport('fiji.plugin.trackmate.features.FeatureFilter')
Table = sj.jimport('org.scijava.table.Table')

In [6]:
# load Java specific resources
Double = sj.jimport('java.lang.Double')

In [7]:
pipeline_params = {
    "time": 30,
    "outradius": 4,
    "outthreshold": 10,
    "outwhich": "Bright", # valid options: "Bright", "Dark"
    "diameter": 20.0,
    "threshold": 0.5,
    "medianfilter": True,
    "subpixel": True,
    "channel": 1,
    "quality": 1.2,
    "link_distance": 15.0,
    "gap_distance": 15.0,
    "gap_frame": 20,
    "max_event_frame": 500,
    "min_event_frames_for_good": 60,
    "event_ext": 0,
    "event_pixel_size": 11,
    "input_dir": "/Users/sdasgupt/Documents/TRD2_protocol/input",
    "output_dir": "/Users/sdasgupt/Documents/TRD2_protocol/output"
}

In [8]:
import scyjava

In [13]:
from ij import IJ, WindowManager, ImagePlus
from ij.io import DirectoryChooser
from ij.gui import WaitForUserDialog, GenericDialog
from ij.plugin import ImageCalculator, PlugIn
from ij.measure import Measurements, ResultsTable
from ij.plugin.frame import RoiManager

ModuleNotFoundError: No module named 'ij'

In [9]:

detection_results = {}
file_type = '.tif'
files = os.listdir(pipeline_params['input_dir'])
for f in files:
    if f[-4:] == file_type:
        name = f.split(".")[0]
        imp = ij.IJ.openImage(os.path.join(pipeline_params['input_dir'], f))
        imp.show()
        ij.IJ.selectWindow(name + file_type)
        
        # get number of frames/slices
        # Assuming 3D data, the first two dimensions in ImageJ are always
        # X and Y, thus the last dimension will be frames/slices
        for i in range(len(imp.dims)):
            if (imp.dims[i] != 'X') and (imp.dims[i] != 'Y'):
                frames = imp.shape[i]

        ij.IJ.run(imp, "Properties...", "channels=1 slices=1 frames=" + str(frames) + " pixel_width=1.0000 pixel_height=1.0000 voxel_depth=1.0000 frame=[" + str(pipeline_params['time']) +" min]")
        ij.IJ.run(imp, "Despeckle", "stack")
        ij.IJ.run("Despeckle", "stack")
        ij.IJ.run("Despeckle", "stack")
        ij.IJ.run(imp, "Remove Outliers...", "radius=" + str(pipeline_params['outradius']) + " threshold=" + str(pipeline_params['outthreshold']) + " which=" + str(pipeline_params['outwhich']) +" stack")
        detection_results['raw_image'] = ij.WindowManager.getImage(name + '.tif').duplicate() # save raw/processed image

        model = Model()
        settings = Settings(imp)
        radius = float(pipeline_params['diameter']) / 2
        settings.detectorFactory = DogDetectorFactory()
        settings.detectorSettings = {
            'DO_SUBPIXEL_LOCALIZATION': pipeline_params['subpixel'],
            'RADIUS': radius,
            'TARGET_CHANNEL': JInt(pipeline_params['channel']),
            'THRESHOLD': float(pipeline_params['threshold']),
            'DO_MEDIAN_FILTERING': pipeline_params['medianfilter']
        }
        filter = FeatureFilter('QUALITY', float(pipeline_params['quality']), True)
        settings.addSpotFilter(filter)
        settings.trackerFactory = SimpleSparseLAPTrackerFactory()
        settings.trackerSettings = LAPUtils.getDefaultLAPSettingsMap()
        settings.trackerSettings['LINKING_MAX_DISTANCE'] = float(pipeline_params['link_distance'])
        settings.trackerSettings['GAP_CLOSING_MAX_DISTANCE'] = float(pipeline_params['gap_distance'])
        settings.trackerSettings['MAX_FRAME_GAP'] = JInt(pipeline_params['gap_frame'])
        tm = TrackMate(model, settings)

        # check TrackMate input and process
        if not tm.checkInput():
            sys.exit(str(tm.getErrorMessage()))

        sm = SelectionModel(model) # needed?
        tm.process()
        use_spots_as_id = True
        export_spots = True
        export_tracks = True
        label_img = LabelImgExporter.createLabelImagePlus(tm, export_spots, export_tracks, use_spots_as_id)
        detection_results['label_image'] = label_img.duplicate() # save label image
        label_img.show()
        ij.IJ.saveAs("Tiff", os.path.join(pipeline_params['output_dir'], name + '_lblImg.tif'))
        ij.IJ.run("Close")

        export_spots = False
        export_tracks = True
        spot_img = LabelImgExporter.createLabelImagePlus(tm, export_spots, export_tracks, use_spots_as_id)
        spot_img.show()
        ij.IJ.saveAs("Tiff", os.path.join(pipeline_params['output_dir'], name + '_SpotImg.tif'))
        ij.IJ.run(imp, "Merge Channels...", "c4=" + name + ".tif c6=" + name + "_SpotImg.tif keep ignore")
        ij.IJ.run(imp, "RGB Color", "frames")
        ij.IJ.saveAs("Tiff",os.path.join(pipeline_params['output_dir'], name + '_merge.tif'))
        ij.IJ.run("Close")

        fm = model.getFeatureModel()
        rt = ij.WindowManager.getWindow("TrackMate Results")

        # table stuff
        if (rt == None) or not (isinstance(rt, TextWindow)):
            table = ResultsTable()
        else:
            table = rt.getTextPanel().getOrCreateResultsTable()
        table.reset

        for id in model.getTrackModel().trackIDs(True):
            v = fm.getTrackFeature(id, 'TRACK_MEAN_SPEED')
            model.getLogger().log('')
            model.getLogger().log('Track ' + str(id) + ': mean velocity = ' + str(v) + ' ' + str(model.getSpaceUnits()) + '/' + str(model.getTimeUnits()))
            track = model.getTrackModel().trackSpots(id)
            for spot in track:
                sid = spot.ID()
                x = spot.getFeature('POSITION_X')
                y = spot.getFeature('POSITION_Y')
                t = spot.getFeature('FRAME')
                q = spot.getFeature('QUALITY')
                model.getLogger().log('\tspot ID = ' + str(sid) + ': x=' + str(x) + ', y=' + str(y) + ', t=' + str(t) + ', q=' + str(q))
                table.incrementCounter()
                table.addValue('TRACK_ID', Double(id))
                table.addValue('SPOT_ID', sid)
                table.addValue('POSITION_X', x)
                table.addValue('POSITION_Y', y)
                table.addValue('FRAME', t)
                table.addValue('QUALITY', q)

        table.show("TrackMate Results")
        ij.WindowManager.getWindow("TrackMate Results").rename(name + '.csv')
        ij.IJ.saveAs("Measurements", os.path.join(pipeline_params['output_dir'], name + '.csv'))
        ij.IJ.selectWindow(name + '.csv')
        ij.IJ.run("Close")
        detection_results['raw_image'].show()
        ij.IJ.selectWindow('DUP_' + name + '.tif')
        ij.IJ.saveAs("Tiff",os.path.join(pipeline_params['output_dir'], name + '_processed.tif'))
        detection_results['tracks'] = ij.py.from_java(ij.convert().convert(table, Table)) # convert ResultsTable to pandas dataframe

Operating in headless mode - the original ImageJ will have limited functionality.
Operating in headless mode - the IJ class will not be fully functional.


java.awt.HeadlessException: java.awt.HeadlessException